# Database parsing

In [ ]:
import pandas, numpy as np
import parsingDatabaseUtils, re, os
import xml, itertools, collections, xml.etree.ElementTree as ET
import tqdm, importlib, dateparser, dateparser.search
import parseMeasurementsByDay
import pickle

In [ ]:
pathDataIds = 'Venezolanas2'
#procedimientosDesc = pandas.read_csv(os.path.join(pathDataIds, 'procedimientosID.csv'), index_col = 0)
#diagnosticos = pandas.read_csv(os.path.join(pathDataIds, 'diagnosis.csv'), index_col = 0)
#medicamentos = pandas.read_csv(os.path.join(pathDataIds, 'medicamentos.csv'), index_col = 0)

In [ ]:
pathFolder = 'Casos2019'
casos = pandas.read_csv(pathFolder + '/casos.csv', index_col = 0)
pacientes = pandas.read_csv(pathFolder + '/pacientes.csv', index_col = 0)
pacientes.index = pacientes.index.map(str)

registros = pandas.read_csv(pathFolder + '/registros.csv', index_col = 0)
registros.index = registros.index.map(str)

#diagnosis = pandas.read_csv('Venezolanas2/diagnosis.csv', index_col = 0)
procedimientos = pandas.read_csv(pathFolder + '/procedimientos.csv', index_col = 0)
procedimientos.index = procedimientos.index.map(str)
registrosByCaso = registros.groupby('Caso')

entriesInfirmery = pandas.read_csv(pathFolder + '/enfermeriaMedidas.csv', index_col = 0)
entriesInfirmeryByCase = entriesInfirmery.groupby('IdAdmision')

In [ ]:
from parsingData.procedures import classificationProcedures

In [ ]:
with open('Data/notFoundInAutomatic.pkl', 'rb') as file:
    errors = pickle.load(file)

## Select a case
Identify all  cases with an associated procedure

In [ ]:
# Identify all  cases with an associated procedure
import difflib
def sameField(et1, et2, field, equal = lambda i, j: i == j):
    return equal(str(et1.find(field).text), str(et2.find(field).text))

def similarTexts(s1, s2, th = .80):
    d =difflib.SequenceMatcher(a = s1, b = s2)
    r = sum((i.size for i in d.get_matching_blocks())) / min(len(s1), len(s2))
    return r > th

def areSameIntervention(r1, r2):
    """
    Decides if they are the same based on the type
    """
    r1ET = ET.fromstring(r1.XmlDescripcion)
    r2ET = ET.fromstring(r2.XmlDescripcion)

    #Date is the same / similar
    isDateSame = sameField(r1ET, r2ET, './/cabecera//fechaCirugia')
    # Type of intervention is the same
    isInterventionSame = sameField(r1ET, r2ET, './/idProcedimiento')

    # Description
    isDescriptionSame = sameField(r1ET, r2ET, './/descripcion', equal = similarTexts)
    field = './/descripcion'
    return isInterventionSame and (isDateSame or  isDescriptionSame)

In [ ]:
interventionToCase = {}
caseToProcedureBirths = {}
checkSeveralProcedures = []
unknownProc = []
procsNotFound = []
for i,r  in  tqdm.tqdm_notebook(registros.loc[registros.CodigoRegistro == 145].iterrows(), total = len(registros)):
    if r.Caso not in casos.index:
        continue
        
    if isinstance(r.RegistroXML, str) and '<row NombreCampo="IdDescripcion"' in r.RegistroXML:
        et = ET.fromstring(r.RegistroXML)
        idDescripcionProcedimiento = et.find('.//row[@NombreCampo="IdDescripcion"]').get('ValorCampo')
        if idDescripcionProcedimiento not in procedimientos.index:
            print('Procedimiento asociado con caso %s no se encuentra (# %s)' % (r.Caso, idDescripcionProcedimiento))
            continue
        interventionToCase[idDescripcionProcedimiento] = (r.Caso, r.NumeroHistoria, i)
        procedureType = re.findall('<idProcedimiento>([a-zA-Z0-9]*)</idProcedimiento>', procedimientos.loc[idDescripcionProcedimiento].XmlDescripcion)[0]
        try:
            typeProc = classificationProcedures[procedureType]
        except:
            typeProc = 'unknown'
            unknownProc.append(procedureType)
        if typeProc == 'p':
            
            #If it is the case that they need to stop labour because a c-section is needed
            #if '<postDiagnosticoPrincipal>O821 - PARTO POR CESAREA DE EMERGENCIA</postDiagnosticoPrincipal>' in \
            #    procedimientos.loc[idDescripcionProcedimiento].XmlDescripcion:
            #    print('continue')
            #    continue
                
            if r.Caso in caseToProcedureBirths:
                r1 = caseToProcedureBirths[r.Caso]
                r2 =procedimientos.loc[idDescripcionProcedimiento]
                if areSameIntervention(r1, r2):
                    if r1.FechaDescripcion < r2.FechaDescripcion:
                        caseToProcedureBirths[r.Caso] = r2
                else:
                    checkSeveralProcedures.append((r.Caso, caseToProcedureBirths[r.Caso],procedimientos.loc[idDescripcionProcedimiento] ))
                    procedureType2 = re.findall('<idProcedimiento>([a-zA-Z0-9]*)</idProcedimiento>', caseToProcedureBirths[r.Caso].XmlDescripcion)[0]

                    print('error, varios procedimientos en caso: ', r.Caso, procedureType, procedureType2)
                    if r1.FechaRegistro > r2.FechaRegistro:
                        caseToProcedureBirths[r.Caso] = procedimientos.loc[idDescripcionProcedimiento]
            else:
                caseToProcedureBirths[r.Caso] = procedimientos.loc[idDescripcionProcedimiento]

In [ ]:
import datetime
birthCases = casos.loc[caseToProcedureBirths.keys()]
birthCases['dayOperation'] = birthCases.FechaHora.map(lambda s: datetime.datetime.strptime(s.split()[0], '%Y-%m-%d') if s == s else s)
birthCasesByDay = birthCases.groupby('dayOperation')

In [ ]:
res = {}
for c in caseToProcedureBirths:
    if c in casos.index and str(casos.loc[c].Paciente) in pacientes.index:
        res[c] =  pacientes.loc[str(casos.loc[c].Paciente)].Identificacion


In [ ]:
cF = 0
cNF  = 0
incorrectId = []
for id, i, date in errors:
    i = i.strip()
    if id[-1] != '0':
        continue
    if i  in res.values() or 'VEN' + i  in res.values() or 'V' + i  in res.values():
        cF += 1
    elif i.replace('V', 'VEN') in res.values()  or i.replace('VEN', "V") in res.values():
        cF += 1
    elif i not in pacientes.index:
        if date in birthCasesByDay.groups:
            incorrectId.append((i, date))
    else:
        print(i)
        cNF += 1
print(cF, cNF)

In [ ]:
cor = 0
incor = 0
for i, date in incorrectId:
    for p in pacientes.loc[map(lambda i: str(int(i)),birthCasesByDay.get_group(date).Paciente)].Identificacion:
        if similarTexts(i, p, th= .75):
            cor += 1
            print(i, p)
            break
    else:
        incor += 1
print(cor, incor)

In [ ]:
for c, r1, r2 in checkSeveralProcedures:
    if areSameIntervention(r1, r2):
        print (c, 'same')
    else:
        print(c, 'Different')


In [ ]:
import birthDatasetStructure
importlib.reload(birthDatasetStructure)
# Split in newborn
registrosByCaso = registros.groupby('Caso')
registrosByPatient = registros.groupby('NumeroHistoria')

twins = {}
processedDatasets = {}
countCaseNotFound = 0
countPatientNotFound = 0

for c, p in tqdm.tqdm_notebook(caseToProcedureBirths.items()):
    if c not in casos.index:
        print('ERROR! Caso no encontrado', c)
        countCaseNotFound += 1
        continue
        
    
    if p.IdPaciente not in registrosByCaso.get_group(c).NumeroHistoria.values:
        print('ERROR! Pacientes inconsistente en caso', c, p.IdPaciente)
        countPatientNotFound += 1
        continue

    if str(p.IdPaciente) not in pacientes.index:
        print('ERROR! Pacientes no encontrado', p.IdPaciente)
        countPatientNotFound += 1
        break
        continue
     
    if c in entriesInfirmeryByCase.groups:
        entriesInf = entriesInfirmeryByCase.get_group(c)
    else:
        entriesInf = pandas.DataFrame()
    
    data = birthDatasetStructure.BirthDataset(c, casos.loc[c], p, registrosByCaso.get_group(c), 
                                                              pacientes,entriesInf)
    if data.epicrisis is None:
        continue
    for i, r in registrosByPatient.get_group(data.motherData.name if 'v' in data.motherData.name .lower() else int(data.motherData.name)).iterrows():
        inRange = lambda d1, d2: (datetime.datetime.strptime(d1, '%Y-%m-%d %H:%M:%S.%f') - datetime.datetime.strptime(d2, '%Y-%m-%d %H:%M:%S.%f')) < datetime.timedelta(days = 7)
        if r.CodigoRegistro == 20:
            if r.FechaAsignacionRegistro < data.epicrisis.FechaAsignacionRegistro and inRange(data.epicrisis.FechaAsignacionRegistro, r.FechaAsignacionRegistro):
                data.registersMother.append(r)
    #Discard twins:
    #if len(data.registrosRecienNacido) > 1:
    #    twins[c] = data
    #    continue
    processedDatasets[c] = data

In [ ]:
#Look for twins
wrongRegisters = []
twins = {}
print('Errors ! More than one register found')
for c, p in processedDatasets.items():
    if any((r  in getAllDiagnosis(p.epicrisis) for r in ['Z372', 'O300'])) :
        #Twins  / 
        #print(len(p.registrosRecienNacido), getAllDiagnosis(p.epicrisis))
        twins[c] = p
        continue
    elif len(p.registrosRecienNacido) > 1 and  any((r  in getAllDiagnosis(p.epicrisis) for r in ['Z370'])) :
        wrongRegisters.append(c)
        print(c, getAllDiagnosis(p.epicrisis),  len(p.registrosRecienNacido))
        continue
    elif len(p.registrosRecienNacido) > 1:
        for k in p.registrosRecienNacido:
            if 'GEMEL' in parsingDatabaseUtils.findInXML('TexTarea_MotivoConsulta', p.registrosRecienNacido[k][k].RegistroXML):
                twins[c] = p
                break
        else:
            print(c, getAllDiagnosis(p.epicrisis),  len(p.registrosRecienNacido))        
            wrongRegisters.append(c)


In [ ]:
for c,data in twins.items():
    if len(data.registrosRecienNacido) != 2:
        continue
    for i, v  in enumerate(data.splitTwins()):
        processedDatasets[c + '-%d' % i] = v


In [ ]:
for c in wrongRegisters:
    del processedDatasets[c]
for c in twins:
    del processedDatasets[c]

In [ ]:
import caseTransformer
importlib.reload(caseTransformer)
importlib.reload(caseTransformer.parseNewbornRegister)
importlib.reload(caseTransformer.parseEpicrisis)
importlib.reload(caseTransformer.parseEchos)
importlib.reload(caseTransformer.parsingDatabaseUtils)
importlib.reload(caseTransformer.parseMeasurementsByDay)
importlib.reload(caseTransformer.parseInterventions)
importlib.reload(caseTransformer.parseDiagnosis)
importlib.reload(caseTransformer.parseMedicaments)

In [ ]:
resSIP = {}


In [ ]:
pp(d.epicrisis.RegistroXML)

In [ ]:
for c, d in tqdm.notebook.tqdm(processedDatasets.items()):
    if c in resSIP:
        continue
    cT = caseTransformer.CaseTransformer(data = d)    
    resSIP[c] = cT.processAll()

In [ ]:
%debug

In [ ]:
pp = parsingDatabaseUtils.prettyPrintXML
pp(list(processedDatasets.values())[500].procedure.XmlDescripcion)

In [ ]:
data = list(processedDatasets.values())[660]

In [ ]:
pp(data.getNewbornRegister(0).RegistroXML)

In [ ]:
import math
def relErrorInLimits(v1, v2, th = .05):
    try:
        v1 = float(v1)
        v2 = float(v2)
        relErr =  2 * math.fabs(v1 - v2)/(v1 + v2)
        return relErr < th
    except:
        return False
def joinDicts(d1, d2):
    d1 = d1.copy()
    d2 = d2.copy()
    for k in d1:
        if k in d2 and d1[k] != d2[k]:
            if relErrorInLimits(d1[k], d2[k]):
                d1[k] = (float(d1[k]) + float(d2[k]))/2
            else:
                d2['error_key_%s' % str(k)] = (True, (d1[k], d2[k]))
    d2.update(d1)
    return d2

In [ ]:
def dateDifferenceDays(d1, d2):
    p1 = parsingDatabaseUtils.parseDate(d1, ''datetime) if isinstance(d1, str) else d1
    p2 = parsingDatabaseUtils.parseDate(d2, 'datetime') if isinstance(d2, str) else d2
    return (p1 - p2).days

def egByEcho(date, res):
    bestDiff = 365
    bestEg  = 0
    date =parsingDatabaseUtils.parseDate(date, True)
    for i in range(10):
        if 'echo_%d_eg' % i in res:
            eg = res['echo_%d_eg' % i] 
            dateEcho = parsingDatabaseUtils.parseDate(res['echo_%d_date' % i], True)
            
            if np.abs(dateDifferenceDays(date, dateEcho)) < bestDiff:
                bestDiff = np.abs(dateDifferenceDays(date, dateEcho))
                bestEg = float(eg) * 7 + dateDifferenceDays(date, dateEcho)
        else:
            break
    return bestEg
 
def dateToGestationDays(date, res):
    """
    Checks wether the event is before or after labour.
    
    - If it is before, return the number of gestation days
    - Otherwise, the time before.
    """
    #If FUM
    
    #Otherwise, use the closest echo
    if True:
        pass
    else:
        days = egByEcho(date, res)
    return days

In [ ]:
for c, p in tqdm.notebook.tqdm(processedDatasets.items()):
    if p.epicrisis is not None and len(p.registrosRecienNacido) == 0:
        print(c)
        


In [ ]:
registros.FechaAsignacionRegistro

In [ ]:
import parseEpicrisis
importlib.reload(parsingDatabaseUtils)
importlib.reload(parseEpicrisis)




resSIP = {}
breakLoop = False
count = 0
for c, p in tqdm.notebook.tqdm(processedDatasets.items()):
    if len(p.registrosRecienNacido) > 1:
        print(c, 'Warning! I still cannot handle twins')
        continue
    resSIP[c] = parseEpicrisis.getMotherData(p)
    resSIP[c] = joinDicts(resSIP[c], parseEpicrisis.getDataFromHospitalAdmision(p))
    resSIP[c] = joinDicts(resSIP[c], parsingDatabaseUtils.getInformationFromProcedureDescription(p))
    for k in p.registrosRecienNacido:
        resSIP[c] = joinDicts(resSIP[c], parsingDatabaseUtils.getNewbornData(p, k))
    if len(p.registrosRecienNacido) == 1:
        resSIP[c]['VAR_0286'] = '0'
        resSIP[c]['VAR_0285'] = 'A'
df = pandas.DataFrame.from_dict(resSIP, orient = 'index')
#l = list(df[(df.VAR_0294 != df.VAR_0294) & (df.VAR_0293 != df.VAR_0293)].index)
#print(len(l))
print(count)

In [ ]:
importlib.reload(parseEpicrisis)
antececedentesText = parsingDatabaseUtils.findInXML('AntecedentesHTML', processedDatasets[k].epicrisis.RegistroXML)

parseEpicrisis.parseEchographies(antececedentesText, cleanText = True)

In [ ]:
import datetime
def toInt(s, d = 1):
    try:
        s = str(int(s))
        return addZeros(s, d)
    except:
        return "X"
def addZeros(s, d):
    s = str(s)
    return '0' * (d - len(str(s))) + s 

In [ ]:
preanestesico, without = 0,0
anesthesia = []
registrosPreanestesico = registros.loc[registros.CodigoRegistro ==20]
for c, p in processedDatasets.items():
    if p.ingreso is None or True:
        hClinica = p.epicrisis.NumeroHistoria
        if len(registrosPreanestesico.loc[registrosPreanestesico.NumeroHistoria == hClinica]):
            preanestesico += 1
            anesthesia.append(registrosPreanestesico.loc[registrosPreanestesico.NumeroHistoria == hClinica].iloc[0])
        else:
            without += 1
preanestesico, without

In [ ]:
keys = list(processedDatasets.keys())
parsingDatabaseUtils.prettyPrintXML(getNewbornRegister(processedDatasets[keys[240]], 0).RegistroXML)

In [ ]:
parseEpicrisis.parseGPCA_and_fum(getNewbornRegister(processedDatasets[c], 0).RegistroXML.lower())

In [ ]:
import parseMeasurementsByDay
importlib.reload(parseMeasurementsByDay)
importlib.reload(parsingDatabaseUtils)
measurements = {}
measurementsControlsPrenatal = {}

def dateDifferenceDays(d1, d2):
    p1 = parsingDatabaseUtils.parseDate(d1, ''datetime) if isinstance(d1, str) else d1
    p2 = parsingDatabaseUtils.parseDate(d2, 'datetime') if isinstance(d2, str) else d2
    return (p1 - p2).days

def processPrenatalMeasurements(measurementsControlsPrenatal, fum):
    res = {}
    for test, date, value, _ in measurementsControlsPrenatal[c]:
        if test in ['vdrl', 'HB', 'prt']:
            weeks = dateDifferenceDays(date, fum)//7
            if weeks < 0:
                raise ValueError('Inconsistent!')

            elif weeks <= 20:
                if test == 'vdrl':
                    #no treponemica
                    res['VAR_0112'] = 'A' if not value else 'B'
                    res['VAR_0412'] = weeks # Weeks

                elif test == 'prt':
                    #treponemica
                    res['VAR_0415'] = 'A' if not value else 'B'
                    res['VAR_0413'] = weeks # Weeks

                elif test == 'HB':
                    resSIP[c]['VAR_0095'] = value

            elif weeks > 20 and weeks < 60:
                if test == 'vdrl':
                    res['VAR_0114'] = 'A' if not value else 'B'
                    res['VAR_0419'] = weeks # Weeks

                elif test == 'prt':
                    res['VAR_0421'] = 'A' if not value else 'B'
                    res['VAR_0420'] = weeks # Weeks

                elif test == 'HB':
                    res'VAR_0099'] = value
    return res

for c, p in tqdm.tqdm_notebook(processedDatasets.items()):
    measurements[c] = parseMeasurementsByDay.getParaClinicsHospitalisation(p) + parseMeasurementsByDay.getAllVitalSigns(p)
    if p.epicrisis is not None:
        measurementsControlsPrenatal[c] = parseMeasurementsByDay.parseParaclinicsBeforeHospitalisation(p.epicrisis)
        fum = resSIP[c]['VAR_0057']
        
        for test, date, value, _ in measurementsControlsPrenatal[c]:
            if test in ['vdrl', 'HB', 'prt']:
                weeks = dateDifferenceDays(date, fum)//7
                if weeks < 0:
                    raise ValueError('Inconsistent!')

                elif weeks <= 20:
                    if test == 'vdrl':
                        #no treponemica
                        resSIP[c]['VAR_0112'] = 'A' if not value else 'B'
                        resSIP[c]['VAR_0412'] = weeks # Weeks

                    elif test == 'prt':
                        #treponemica
                        resSIP[c]['VAR_0415'] = 'A' if not value else 'B'
                        resSIP[c]['VAR_0413'] = weeks # Weeks

                    elif test == 'HB':
                        resSIP[c]['VAR_0095'] = value

                elif weeks > 20:
                    if test == 'vdrl':
                        resSIP[c]['VAR_0114'] = 'A' if not value else 'B'
                        resSIP[c]['VAR_0419'] = weeks # Weeks

                    elif test == 'prt':
                        resSIP[c]['VAR_0421'] = 'A' if not value else 'B'
                        resSIP[c]['VAR_0420'] = weeks # Weeks

                    elif test == 'HB':
                        resSIP[c]['VAR_0099'] = value


In [ ]:
for c, p in tqdm.tqdm_notebook(processedDatasets.items()):
    if len(p.registrosRecienNacido) > 1:
        print('Warning! I still cannot handle twins')
        continue

    res = {}
    dfMeasurements = pandas.DataFrame(data = measurements[c], columns =['Campo', 'Fecha', 'Valor'])
    dfMeasurements.Valor = dfMeasurements.Valor.map(lambda s: s.replace(',', '.') if isinstance(s, str) else s)
    dfMeasurements.Valor = dfMeasurements.Valor.astype(float)
    dfMeasurementsByDate = dfMeasurements.groupby('Fecha')
    for i, d in enumerate(dfMeasurementsByDate.groups):
        dfValues =dfMeasurementsByDate.get_group(d).groupby('Campo')['Valor'].agg(['median', 'max', 'min'])
        res['day_%d' % i] = d
        for var, row in dfValues.iterrows():
            res[var + '_median_day_%d' % i] = row.median()
            res[var + '_max_day_%d' % i] = row.max()
            res[var + '_min_day_%d' % i] = row.min()
    resSIP[c].update(res)

In [ ]:
from parsingData import parseDiagnosis
from parsingData import parseInterventions, parseMedicaments, parseWithDictionary

importlib.reload(parseDiagnosis)
for c, p in tqdm.tqdm_notebook(processedDatasets.items()):
    if len(p.registrosRecienNacido) > 1:
        print('Warning! I still cannot handle twins')
        continue
    s = getAllDiagnosis(p.epicrisis)
    s = [parsingDatabaseUtils.remove_diacritics(pars.unescape(ss)) for ss in s]
    resSIP[c].update(parseWithDictionary.parseWithDictionary(s, parseDiagnosis.actionsDiagnosis))

In [ ]:
from parsingData import parseInterventions, parseMedicaments, parseWithDictionary
importlib.reload(parseMedicaments)
importlib.reload(parseInterventions)

#Parse medicaments and interventions
for c, p in tqdm.tqdm_notebook(processedDatasets.items()):
    if len(p.registrosRecienNacido) > 1:
        print('Warning! I still cannot handle twins')
        continue
    m, i = getMedicamentsInterventionsFromEpicrisis(p.epicrisis)
    for k, v in m.items():
        v = [vv[0] for vv in v]
        resSIP[c].update(parseWithDictionary.parseWithDictionary(v, parseMedicaments.actionMedicament))
        
    for k, v in i.items():
        v = [vv[0] for vv in v]
        resSIP[c].update(parseWithDictionary.parseWithDictionary(v, parseInterventions.actionIntervention))

In [ ]:
dfResSIP = pandas.DataFrame.from_dict(resSIP, orient = 'index')

#Quality control
n = len(dfResSIP)
for c in dfResSIP.columns:
    if sum(dfResSIP[c] != dfResSIP[c])/n > .01 and not c.startswith('error'):
        print(c, sum(dfResSIP[c] != dfResSIP[c])/n )

In [ ]:
dfResSIP = pandas.DataFrame.from_dict(resSIP, orient = 'index')

#Add SIP information
placeId = '806001061-8'
placeId = addZeros(placeId, 20)
dfResSIP['ID01'] = placeId + dfResSIP['VAR_0019'].map(lambda s: addZeros(s,20))  \
                                          + dfResSIP['VAR_0040'].map(lambda s: toInt(s, 2)) + \
                                        dfResSIP['VAR_0286'].map(toInt)
dfResSIP['VAR_0198'] = df['VAR_0198'].map(lambda s: str(int(s)) if s == s and s else s)
dfResSIP['VERSION'] = '4.1.2'
dfResSIP['FECHA'] = datetime.datetime.now().strftime("%Y-%m-%d")
dfResSIP['HORA'] = datetime.datetime.now().strftime("%H:%M:%S")
dfResSIP['USUARIO'] = 'CMRC-AUTO'

dfResSIP.to_csv('Output/results.csv')

In [ ]:
dfResSIP.columns

In [ ]:
dfNamesVars = pandas.read_excel('parsingData/variables_sip.xlsx')
renameDict = {k : v for  k, v in zip(dfNamesVars['Unnamed: 2'].values, dfNamesVars['Unnamed: 3'].values)}

dfResSIP.columns = dfResSIP.columns.map(lambda s: s if s not in renameDict else renameDict[s])

In [ ]:
groupedBy = dfNamesVars.groupby('Unnamed: 2')
for g in groupedBy.groups:
    if len(groupedBy.get_group(g)) == 1 or renameDict[g] not in dfResSIP:
        continue
    colname = renameDict[g]
    for (_, r) in groupedBy.get_group(g)[['Unnamed: 4', 'Unnamed: 5']].iterrows():
        option = r['Unnamed: 4']
        value = r['Unnamed: 5']
        dfResSIP.loc[dfResSIP[colname] == option, colname] = value

In [ ]:
dfResSIP.to_csv('Output/resultsPrettyName.csv')

# Tests

From here on, it is 

In [ ]:
allMedicaments = {}
allInterventions = {}
for c, p in tqdm.tqdm_notebook(processedDatasets.items()):
    m, i = getMedicamentsInterventionsFromEpicrisis(p.epicrisis)
    
    for _, v in m.items():
        for c, desc, _ in v:
            allMedicaments[c] = desc
            
    for _, v in i.items():
        for c, desc, _ in v:
            allInterventions[c] = desc

In [ ]:
for k, v in allInterventions.items():
    if k not in parseInterventions.actionIntervention:
        print('actionIntervention["%s"] = "" #%s' % (k, parsingDatabaseUtils.remove_diacritics(v)))


In [ ]:
with open('parseMedicaments.py', 'w') as f:
    f.write('actionMedicament = {} \n')
    for k, v in allMedicaments.items():
        f.write('actionMedicament["%s"] = "" #%s\n' % (k, parsingDatabaseUtils.remove_diacritics(v)))
        
with open('parseInterventions.py', 'w') as f:
    f.write('actionIntervention = {} \n')
    for k, v in allInterventions.items():
        f.write('actionIntervention["%s"] = "" #%s\n' % (k, parsingDatabaseUtils.remove_diacritics(v)))

In [ ]:
parseMedicamentsInterventionsByDay(parsingDatabaseUtils.findInXML('MedicamentosAdministrado', et))

In [ ]:
et = ET.fromstring(p.epicrisis.RegistroXML)
parsingDatabaseUtils.findInXML('MedicamentosAdministrado', et)
print(parsingDatabaseUtils.findInXML('ProcedimientosAsociados', et))

In [ ]:
import parseDiagnosis

In [ ]:
allDiagnosis = set()
for c, p in tqdm.tqdm_notebook(processedDatasets.items()):
    allDiagnosis.update(set(getAllDiagnosis(p.epicrisis)))


In [ ]:
allDiagnosis = set(list(allDiagnosis) + list(parseDiagnosis.actionsDiagnosis.keys()))

In [ ]:
for a in sorted(allDiagnosis):
    #if a in actionsDiagnosis:
    #    pass
    if a not in diagnosticos.index:
        continue 
    if False and a in parseDiagnosis.actionsDiagnosis:
        print("actionsDiagnosis['%s'] = '%s' #%s"% (a,parseDiagnosis.actionsDiagnosis[a], diagnosticos.loc[a, 'dfg']))
    else: 
        print("actionsDiagnosis['%s'] = '' #%s"% (a,diagnosticos.loc[a, 'dfg']))


In [ ]:
parsingDatabaseUtils.prettyPrintXML(p.epicrisis.RegistroXML)

In [ ]:
[p.epicrisis.Diagnostico, p.epicrisis.CodigoDiagnosticoRelacionado1,  p.epicrisis.CodigoDiagnosticoRelacionado2, p.epicrisis.CodigoDiagnosticoRelacionado3] 

In [ ]:
for c,  _ in df[df['sufrimientoFetal'] == 'SI'].iterrows():
    et = ET.fromstring(processedDatasets[c].epicrisis.RegistroXML)
    print(parsingDatabaseUtils.findInXML('MedicamentosAdministrado', et ))

In [ ]:
noEpicrsis = []
noIngreso = []

noDischarge = []
for c, p in processedDatasets.items():
    if p.epicrisis is None:
        noDischarge.append(c)
    if p.ingreso is None:
        noIngreso.append(c)
print('no emergency' , len(noIngreso), 'no epi', len(noDischarge), len(processedDatasets))

In [ ]:
pacientes.loc[str(casos.loc[noIngreso[0], 'Paciente'])], noIngreso[0], casos.loc[noIngreso[0]]

In [ ]:
#count = 0
#for c, p in processedDatasets.items():
#    if p.epicrisis is None:
#        continue
#    et = ET.fromstring(p.epicrisis.RegistroXML)
#    ant = parsingDatabaseUtils.fullCleanTxt(parsingDatabaseUtils.findInXML('AntecedentesHTML', et))
#    if not 'paracli' in ant:
#        count += 1
#    else:
#        print(ant)

In [ ]:

(dateparser.parse('6/1/20') - dateparser.parse('4/11/19')).days // 7, (dateparser.parse('6/1/20') - dateparser.parse('4/11/19')).days %7

In [ ]:
def dateDifferenceDays(d1, d2):
    p1 = dateparser.parse(d1) if isinstance(d1, str) else d1
    p2 = dateparser.parse(d2) if isinstance(d2, str) else d2
    return (p1 - p2).days

def egByEcho(date, res):
    bestDiff = 365
    bestEg  = 0
    date =dateparser.parse(date)
    for i in range(10):
        if 'echo_%d_eg' in res:
            eg = res['echo_%d_eg' % i] 
            dateEcho = dateparser.parse(res['echo_%d_date' % i])
            
            if np.abs(dateDifferenceDays(date, dateEcho)) < bestDiff:
                bestDiff = np.abs(dateDifferenceDays(date, dateEcho))
                bestEg = float(eg) * 7 + dateDifferenceDays(date, dateEcho)
    return bestEg
 
def dateToGestationDays(date, res):
    """
    Checks wether the event is before or after labour.
    
    - If it is before, return the number of gestation days
    - Otherwise, the time before.
    """
    #If FUM

    #Otherwise, use the closest echo
    else:
        days = egByEcho(date, res)
    return days

In [ ]:
c = 'AD284225'
k = next(iter(processedDatasets[c].registrosRecienNacido.keys()))
parsingDatabaseUtils.prettyPrintXML(processedDatasets[c].registrosRecienNacido[k][k].RegistroXML)


In [ ]:
c = list(processedDatasets.keys())[884]
count = 0
for c in processedDatasets.keys():
    try:
        if 'ECLAMPS' in processedDatasets[c].epicrisis.RegistroXML:
            count += 1
            #parsingDatabaseUtils.prettyPrintXML(processedDatasets[c].epicrisis.RegistroXML)
    except:
        pass

In [ ]:
df.to_csv('test.csv')

In [ ]:
c = list(processedDatasets.keys())[244]
parsingDatabaseUtils.prettyPrintXML(processedDatasets[c].epicrisis.RegistroXML)

In [ ]:
import cProfile

In [ ]:
paraClinics = {}
for c, p in tqdm.tqdm_notebook(processedDatasets.items()):
    if p.epicrisis is None:
        continue
    d =parseParaclinics(p.epicrisis)
    paraClinics[c] = paraclinicsToDF(parseParaclinics(p.epicrisis))


In [ ]:
dfParaClinics.noParaclinicalTestsConfirmed.value_counts()

In [ ]:
dfParaClinics = pandas.DataFrame.from_dict(paraClinics, orient = 'index')
dfParaClinics = dfParaClinics[sorted(dfParaClinics.columns)]
dfParaClinics.to_csv('paraclinics.csv')

In [ ]:
for p, d in processedDatasets.items():
    break

In [ ]:
getMedicamentsAndpProcedures(d.epicrisis)

In [ ]:
from html.parser import HTMLParser
pars =HTMLParser()

def splitByDate(txt):
    txt = pars.unescape(txt)
    txtByDate = txt.split('Fecha:')
    txtByDate = map(lambda s: s.strip(), txtByDate)
    txtByDate = {m.split()[0] : m for m in txtByDate if m}
            print(m)

    processedByDate = {}
    for v,m in txtByDate.items():
        processedByDate[v] =  re.findall('-([0-9a-zA-Z\- ]+) - (.+?)- Cantidad: ([0-9]+)', m, re.MULTILINE)
    return processedByDate
def getMedicamentsAndpProcedures(epicrisis):
    et = ET.fromstring(epicrisis.RegistroXML)
    medicamentos = parsingDatabaseUtils.findInXML('MedicamentosAdministrado' , et)
    procedimientos = parsingDatabaseUtils.findInXML('ProcedimientosAsociados' , et)
    #return medicamentos, procedimientos
    return splitByDate(medicamentos), splitByDate(procedimientos)

In [ ]:
parsingDatabaseUtils.prettyPrintXML(d.epicrisis.RegistroXML)

In [ ]:
for c, p in paraClinics.items():
    if len(p) != 1:
        print(paraClinics[c]['text'])
        print('-----------------')
        for k in p:
            if k != 'text':
                print(k, p[k])
        print('-----------------')

In [ ]:
def merge2DF(df1, df2):
    """
    Returns the merge of 2 df with the same index (1 to 1)
    In case of columns with the same name, check if the values are the same (or one of them is NA)
    Creates new error column to see where there has been problems
    """
    pass

# Check failures

In [ ]:
import dateparser

In [ ]:
str(dateparser.parse('4/4/17'))

In [ ]:
epi = processedDatasets[list(processedDatasets.keys())[672]].epicrisis

In [ ]:
antecedentes = parsingDatabaseUtils.findInXML('AntecedentesHTML', epi.RegistroXML)

In [ ]:
importlib.reload(parsingDatabaseUtils)
parsingDatabaseUtils.parseEchographies(antecedentes, cleanText=True)

In [ ]:
re.findall(parsingDatabaseUtils.date, '6 de marzo de 1995', re.IGNORECASE)

In [ ]:
import importlib
importlib.reload(parsingDatabaseUtils)
pp = parsingDatabaseUtils.prettyPrintXML

In [ ]:
registros.loc[registros.Caso == 'AD348509']